In [ ]:
PORTAL_PORT=49051
PORTAL_HOST='0.0.0.0'
DATABASE_PORT=27017

import datetime
from bson.objectid import ObjectId
import random
import string
import hmac
import hashlib
from hashlib import sha256
from flask import Flask
from flask import request,redirect
from flask import Response
from flask import render_template
from os import listdir
from os.path import isfile, join
import os
from pymongo import MongoClient
import pymongo
import os.path

if os.path.isfile('.ADMIN_PASSWORD'):
    with open('.ADMIN_PASSWORD','r') as f:
        ADMIN_PASSWORD = f.read()
else:
    with open('.ADMIN_PASSWORD','w') as f:
        ADMIN_PASSWORD = "ADMIN_PASSWORD"
        f.write(ADMIN_PASSWORD)


app = Flask(__name__)
app.config['TEMPLATES_AUTO_RELOAD'] = True

@app.after_request
def add_header(r):
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r

@app.route('/create')
def create():
    return render_template("formCreate.html")

@app.route('/createError')
def createError():
    return render_template("createError.html")

@app.route('/saveError')
def saveError():
    return render_template("saveError.html")

@app.route("/createSave",methods=['POST'])
def createSave():
    if request.form.get('name', "")=="":
        return redirect("/createError")
    if request.form.get('email', "")=="":
        return redirect("/createError")
    if request.form.get('devName', "")=="":
        return redirect("/createError")
    if len(request.form.get('appPassword', ""))<10:
        return redirect("/createError")
    form_text=["name","desc","email","devName"]
    form_checks=["spisok","read","write","delete"]
    appObject={i:request.form.get(i, "") for i in form_text}
    for i in form_checks:
        t=request.form.get(i, "off")
        if t=="on":
            t="zapros"
        appObject[i]=t
    appObject["salt"]=''.join(random.choices(string.ascii_letters + string.digits, k=10))
    appObject["passwordHash"]=sha256((appObject["salt"]+request.form.get('appPassword', "")).encode('utf-8')).hexdigest()
    appObject["secretKey"]=sha256(''.join(random.choices(string.ascii_letters + string.digits, k=10)).encode('utf-8')).hexdigest()
    appObject["date"]=datetime.datetime.utcnow()
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    result=db.apps.insert_one(appObject)
    link=db.links.insert_one({"appID":result.inserted_id})
    return redirect(f"/secretKey?link_id={link.inserted_id}")

@app.route("/getApp",methods=['GET'])
def getApp():
    appID=request.args.get('id',"Ошибка")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    return render_template("getApp.html",appObject=appObject)

@app.route("/",methods=['GET'])
def appList():
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    apps=list()
    for i in db.apps.find({}).sort("date",pymongo.DESCENDING):
        t=i
        m=["spisok","read","write","delete"]
        status="off"
        for i2 in m:
            if i[i2] == "on":
                status="on"
        for i2 in m:
            if i[i2] == "zapros":
                status="zapros"
        t["status"]=status
        apps.append(t)
    return render_template("appList.html",apps=apps,mode="read")
    
@app.route("/admin",methods=['GET'])
def admin():
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    apps=list()
    for i in db.apps.find({}).sort("date",pymongo.DESCENDING):
        t=i
        m=["spisok","read","write","delete"]
        status="off"
        for i2 in m:
            if i[i2] == "on":
                status="on"
        for i2 in m:
            if i[i2] == "zapros":
                status="zapros"
        t["status"]=status
        apps.append(t)
    return render_template("appList.html",apps=apps,mode="admin")

@app.route("/adminApp")
def adminApp():
    appID=request.args.get('id',"Ошибка")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    return render_template("adminApp.html",appObject=appObject)

@app.route("/adminSave",methods=['POST'])
def adminSave():
    print(request.form)
    if request.form.get('name', "")=="":
        return redirect("/saveError")
    if request.form.get("adminPassword","") != ADMIN_PASSWORD:
        return redirect("/saveError")
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.update_one({"_id":ObjectId(appID)},
                       {'$set': 
                       {
                           "name":request.form.get('name', ""),
                           "desc":request.form.get('appDesc', ""),
                           "devName":request.form.get('devName', ""),
                           "email":request.form.get('email', ""),
                           "spisok":request.form.get('spisok', ""),
                           "read":request.form.get('read', ""),
                           "write":request.form.get('write', ""),
                           "delete":request.form.get('delete', ""),
                       }})
    return redirect(f"/adminApp?id={appID}")

@app.route("/adminDelete",methods=['POST'])
def adminDelete():
    if request.form.get("adminPassword","") != ADMIN_PASSWORD:
        return redirect("/saveError")
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.delete_one({"_id":ObjectId(appID)})
    return redirect("/admin")

@app.route('/secretKey')
def secretKey():
    linkID=request.args.get('link_id',"Ошибка")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    linkObject=db.links.find_one({"_id":ObjectId(linkID)})
    appObject=db.apps.find_one({"_id":ObjectId(linkObject["appID"])})
    db.links.delete_one({"_id":ObjectId(linkID)})
    return render_template("secretKey.html",appObject=appObject)

@app.route("/devApp")
def devApp():
    appID=request.args.get('id',"Ошибка")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    return render_template("devApp.html",appObject=appObject)

@app.route("/devNewKey",methods=["POST"])
def devNewKey():
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    salt=appObject["salt"]
    dbHash=appObject["passwordHash"]
    formHash=sha256((salt+request.form.get('devPassword', "")).encode('utf-8')).hexdigest()
    if dbHash != formHash:
        return redirect("/saveError")
    newKey=sha256(''.join(random.choices(string.ascii_letters + string.digits, k=10)).encode('utf-8')).hexdigest()
    db.apps.update_one({"_id":ObjectId(appID)},
                       {'$set': 
                       {
                           "secretKey":newKey
                       }})
    link=db.links.insert_one({"appID":ObjectId(appObject["_id"])})
    return redirect(f"/secretKey?link_id={link.inserted_id}")

@app.route("/devDelete",methods=["POST"])
def devDelete():
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    salt=appObject["salt"]
    dbHash=appObject["passwordHash"]
    formHash=sha256((salt+request.form.get('devPassword', "")).encode('utf-8')).hexdigest()
    if dbHash != formHash:
        return redirect("/saveError")
    newKey=sha256(''.join(random.choices(string.ascii_letters + string.digits, k=10)).encode('utf-8')).hexdigest()
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.delete_one({"_id":ObjectId(appID)})
    return redirect("/")

@app.route("/devChangePassword",methods=["POST"])
def devChangePassword():
    if len(request.form.get('newPassword', ""))<10:
        return redirect("/saveError")
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    salt=appObject["salt"]
    dbHash=appObject["passwordHash"]
    formHash=sha256((salt+request.form.get('devPassword', "")).encode('utf-8')).hexdigest()
    if dbHash != formHash:
        return redirect("/saveError")
    salt=''.join(random.choices(string.ascii_letters + string.digits, k=10))
    passwordHash=sha256((salt+request.form.get('newPassword', "")).encode('utf-8')).hexdigest()
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.update_one({"_id":ObjectId(appID)},
                       {'$set': 
                       {
                           "passwordHash":passwordHash,
                           "salt":salt,
                       }})
    return redirect(f"/getApp?id={appID}")

@app.route("/adminChangePassword",methods=["POST"])
def adminChangePassword():
    if len(request.form.get('newPassword', ""))<10:
        return redirect("/saveError")
    appID=request.form.get('appID', "")
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    if request.form.get("adminPassword","") != ADMIN_PASSWORD:
        return redirect("/saveError")
    salt=''.join(random.choices(string.ascii_letters + string.digits, k=10))
    passwordHash=sha256((salt+request.form.get('newPassword', "")).encode('utf-8')).hexdigest()
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    db.apps.update_one({"_id":ObjectId(appID)},
                       {'$set': 
                       {
                           "passwordHash":passwordHash,
                           "salt":salt,
                       }})
    return redirect(f"/adminApp?id={appID}")
                        
                        
app.run(host=PORTAL_HOST, port=PORTAL_PORT, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:49051/ (Press CTRL+C to quit)
127.0.0.1 - - [03/May/2021 15:28:25] "GET /getApp?id=608f1351b59c6f34109189d9 HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 15:28:44] "GET / HTTP/1.1" 200 -
